# Data processing 

In this project data from the Brussels Mobility Bike Counts API will be processed using PySpark. The devices are sensors located in Brussels to count the number of bikes passed by and measure their speed every 15 minutes. The data that will be used is historical data from 2018-12-06 to 2023-
03-31. This can be done by reading csv from the URL provided.

Importing libraries

In [ ]:
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as F
from pyspark.sql.types import StructField, StringType, StructType, IntegerType, FloatType
import pandas as pd
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import socket
import time

Reading the data from the URL and writng them to csv files in SENSORs_DATA folder
##### Downloading the data
- First, we gather the device names by querying the API, requesting the devices with `?request=devices` for all the required dates.
- Then, we only select the device names and remove duplicates by building a set.

In [ ]:
## Code block to load data

import requests
devices_full_url = 'https://data.mobility.brussels/bike/api/counts/?request=devices&startDate=20181206&endDate=20230331&outputFormat=csv'
devices_full_df = pd.read_csv(devices_full_url)

device_names = set(devices_full_df['Device name'])

# there are 18 devices
k=len(device_names)

# printing devices
devicesFiles=[]
for j in range(k):
  devicesFiles.append('device'+str(j+1)+'history.csv')
print(devicesFiles)

It is seen that there are 18 sensors to be analyzed in the project. To download the data, we build a list from the set and iterate over the list. The aim is to store all the data in a single `data.csv` file. So we first open the file, and then we iterate over all the devices. For each, we make a request through the API and write the result in the big file.

In [ ]:
## Download data
# create data folder
!mkdir SENSORS_DATA
i=0
for device in list(device_names) :
    url = 'https://data.mobility.brussels/bike/api/counts/?request=history&featureID='+device+'&startDate=20181206&endDate=20230331&outputFormat=csv'
    response = requests.get(url)
    lines = response.text.splitlines()
    header = lines[0] + ",Sensor"
    with open(f'SENSORS_DATA/{devicesFiles[i]}', 'w', newline='') as f :
        writer = csv.writer(f)
        
        # The structure of the files is `Date,Time gap,Count,Average speed,Sensor`
        for line in lines[1:] :
            line += f",{device}"
            writer.writerow(line.split(','))
    i += 1

# Filling missing data
It is seen that some sensors have several missing time stamps, so to complete the process they need to be added. Each day has 96 time gaps. A function to flatten the time gap by computing the number of days between 2018-12-06 and 2023-03-31 and setting time gap of 0 to the starting date then adding 96 to each day. By setting the time gap as index then reindexing the data we will have all the missing data and they can filled with count 0 and speed -1.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# FLATTEN TIME GAP
# A pair 'day, time gap' is mapped to an absolute timegap, with timestamp 0 being '2018-12-06, 0'. 
# Each day is 96 time gaps.
# So we first compute the number of days between '2018-12-06' and the current day
# And we multiply this by 96 to have the number of elapsed time gaps
# And we add the time gap of the day, which is the other element in the pair

gaps_per_day = 96
initialDayString = '2018-12-06'
initialDay = datetime.strptime(initialDayString, '%Y-%m-%d')

# Requires tgap to be integer, and 'day' to be a string 
def flatten_timestamp(day, tgap) :
    day_delta = (datetime.strptime(day,'%Y-%m-%d') -initialDay).days
    return tgap + day_delta*gaps_per_day

# Works also in a single line
# flatten_timestamp = lambda tgap : int(tgap[1]) + (datetime.strptime(str(tgap[0]),'%Y-%m-%d') -initialDay).days*gaps_per_day



In [ ]:
# Min and max timestamp
max_timestamp = flatten_timestamp('2023-03-31', 96)
min_timestamp = 1
timestamp_range = range(min_timestamp, max_timestamp)

This code saves a data structure as ttime gab and count for sensor appended to it as the following

- tagp | c1 | ... | c18
- 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
- 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

This will let the stream processing work well and take each row as a record for specific time gap

In [ ]:
!mkdir CLEAN_DATA

In [ ]:
import pandas as pd
big_df = pd.DataFrame()

for deviceFile in devicesFiles :

    print(f"Processing {deviceFile}")

    df = pd.read_csv(f"SENSORS_DATA/{deviceFile}", header=None)
    sensor_name = df[4][0]

    print(f"Dataset for sensor {sensor_name} loaded")

    # Extracting all the time gaps from the dataframe in a single columnar dataframe
    gaps_df = df.apply(lambda x : flatten_timestamp(x[0], x[1]), axis=1)


    # Setting this column as the index of the dataframe
    df.index = gaps_df

    # Removing initial 'day, time gap' columns
    df = df[[2,3,4]]

    # Time to fill missing lines with Pandas's 'reindex' method
    # 'reindex' method can only fill with one value. Choosing 'np.NAN' so we
    # can later fill np.NAN with our values
    df = df.reindex(list(timestamp_range),fill_value=np.NAN)

    # Filling NANs accordingly
    df = df.fillna(value={
        2:0,
        3:-1,
        4:sensor_name})
   

  # renaming the columns as count_senosr_name.....  
    df = df.rename(columns={2: f"count_{sensor_name}"})

    # Right now only keeping the time gap(index) and the count
    df = df[[f"count_{sensor_name}"]]

    # add the new device info to one big df
    if big_df.empty:
        big_df = df
    else:
        big_df = pd.merge(big_df, df, left_index=True, right_index=True, how='outer')

# saving the data to final_data
big_df.head()
big_df.to_csv(f"CLEAN_DATA/final_data.csv", header=True)

## Only run after this cell if you downloaded the data

# Producer notebook

This notebook simulates the producer of streaming data. This is done by reading the data in the CSV files, and at each time interval, sending some data to a server. This requires a connection to the server first. 

We use server name `localhost` and port  to communicate.

The parameters for the notebook are the size of the batch  `pi`, i.e the number of days of data that will be sent each time, and `delta`, the time interval.

In [1]:
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as F
from pyspark.sql.types import StructField, StringType, StructType, IntegerType, FloatType
import pandas as pd
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import socket
import time

## Setting up the data source : Spark DataFrame

In this scenario, we will consider a huge dataframe from which we will send the data. For manipulating teh dataframe, we use `Spark DataFrames`. Thus, we need to setup a Spark session in our producer node.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master("local")\
    .appName("Producer")\
    .getOrCreate()

sc = spark.sparkContext

## Setting up the socket connection

The following block of code sets up the connection to the server. It is to be ran, and expecting **the server to be up and running** (in the `Phase_2_consumer.ipynb` file).

Run the following block and head to the `Phase_2_consumer` file to set up the server.

In [3]:
import socket
  
# take the server name and port name
host = 'localhost'
port = 9999
  
# create a socket at server side
# using TCP / IP protocol
s = socket.socket(socket.AF_INET,
                  socket.SOCK_STREAM)
  
# bind the socket with server
# and port number
s.bind((host, port))
  
# allow maximum 1 connection to
# the socket
s.listen(30)
  
# wait till a client accept
# connection
c, addr = s.accept()
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
  
# display client address
print("CONNECTION FROM:", str(addr))

CONNECTION FROM: ('127.0.0.1', 40806)



## Filtering the data at each stage
Each time the `delta` period is elapsed, a batch of data is emitted and this batch is a subset of the `data` dataframe. This subset is built around the `timegap` value of the rows, for which at each emission we set a lower bound and an upper bound, and all the data between those bounds is selected.

For instance, the bounds for the first iteration are obviously `0` and `gaps_per_day*pi`. For the next iteration, the lower bound becomes the former upper bound `+1`, and the upper bound becomes the current lower bound `+gaps_per_day*pi`.

```python
def getNextBounds(lower, upper, pi) :
    lower = upper 
    upper = lower + pi*gaps_per_day
    return lower, upper
``` 

In [4]:
delta = 5 # seconds
pi = 30 # days
gaps_per_day = 96
gaps = pi*gaps_per_day

def getNextBounds(lower, upper, batch_size=pi) :
    lower = upper
    upper = lower + batch_size*gaps_per_day
    return lower, upper

data = sc.textFile("CLEAN_DATA/final_data.csv")
data = data.map(lambda line : line.split(","))
header = data.first()
data = data.filter(lambda row: row != header)
data

PythonRDD[3] at RDD at PythonRDD.scala:53

# Task 1 and 2 data structure sending
This code sends data in the following form :


- tagp | s1 | ... | s18
- 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
- 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

For Task 1 and 2 the data is sent and processed record by record

In [5]:
# Setting parameters again
# Setting initial bounds
pi = 1 # days
delta = 5 # seconds
lower, upper = 0, pi*96

### Selecting subset of the data
# The data is encoded in an RDD. We can use the `filter` method.
while True :

    restrained_data = data.filter(lambda line : float(line[0]) <= upper and float(line[0]) > lower)
    restrained_data = restrained_data.map(lambda line : ','.join(line))
    message = str('\n'.join(restrained_data.collect())) + '\n'

    # Send message to the client
    try:  
        print("Sending message")
        print(message)
        c.send(message.encode())
    except socket.error:
        # If failed, client is probably disconnected. Wait for another connection
        c.close()
        c, addr = s.accept()

    lower, upper = getNextBounds(lower, upper, batch_size=pi)
    time.sleep(delta)

Sending message
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0

Sending message
193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Sending message
385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Sending message
577,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0
578,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
579,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
580,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
581,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
582,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
583,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
584,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
585,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
586,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
587,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
589,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Sending message
769,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
771,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
772,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
773,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
774,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
777,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
778,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
779,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
781,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,

Sending message
961,2.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
962,3.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0
963,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0
964,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
965,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
966,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
967,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
968,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
969,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
970,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
971,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0
972,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
973,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Sending message
1153,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1154,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1155,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1156,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1157,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1158,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1159,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1161,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1163,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1165,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

KeyboardInterrupt: 

# Task 3 and 4  data structure sending

in task 3 and 4 the entire batch is sent and processed instead of record by record


In [6]:
# Setting parameters again
pi = 1
delta = 5

lower = flatten_timestamp('2022-04-15', 96) # Cf. assignment
lower = 0                                   # to take data since 2018
upper = lower +  pi*96



while True :

    restrained_data = data.filter(lambda line : float(line[0]) <= upper and float(line[0]) > lower)
    restrained_data = restrained_data.map(lambda line : np.array(line).astype(float))
    data_array = np.array(restrained_data.collect())
    
    # To avoid printing Ellipsis ("...") in the string
    np.set_printoptions(threshold = np.prod(data_array.shape))
    
    message = np.array2string(data_array,separator=",").replace("\n", "") + "\n"

    # Send message to the client
    try:  
        print("Sending message")
        print(message)
        c.send(message.encode())
    except socket.error:
        # If failed, client is probably disconnected. Wait for another connection
        c.close()
        c, addr = s.accept()

    lower, upper = getNextBounds(lower, upper, batch_size=pi)
    time.sleep(delta)


Sending message
[[ 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,   0.], [ 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,   0.], [ 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,   0.], [ 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,   0.], [ 5., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,   0.], [ 6., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,   0.], [ 7., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,   0.], [ 8., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,   0.], [ 9., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,   0.], [10., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,   0.], [11., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,   0.], [12., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,   0.], [13., 0., 0

Sending message
[[193.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [194.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [195.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [196.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [197.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [198.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [199.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [200.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [201.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [202.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,

Sending message
[[289.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [290.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [291.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [292.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [293.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [294.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [295.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [296.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [297.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [298.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,

Sending message
[[385.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [386.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [387.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [388.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [389.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [390.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [391.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [392.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [393.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [394.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,

Sending message
[[481.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [482.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [483.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [484.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  1.], [485.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  2.,    0.,  0.,  0.,  0.,  1.], [486.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [487.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  1.], [488.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  2.], [489.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,  0.,  0.,  0.], [490.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,    0.,  0.,

...

Sending message
[[1.441e+03,4.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  2.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,1.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,2.000e+00], [1.442e+03,2.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  2.000e+00,0.000e+00,5.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00], [1.443e+03,1.000e+00,0.000e+00,0.000e+00,2.000e+00,0.000e+00,0.000e+00,  5.000e+00,0.000e+00,4.000e+00,0.000e+00,0.000e+00,0.000e+00,3.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,2.000e+00], [1.444e+03,1.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  4.000e+00,0.000e+00,4.000e+00,0.000e+00,0.000e+00,0.000e+00,1.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00], [1.445e+03,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  1.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,1.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00]

Sending message
[[1.537e+03,3.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  1.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,2.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,1.000e+00], [1.538e+03,6.000e+00,0.000e+00,0.000e+00,1.000e+00,0.000e+00,0.000e+00,  9.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,3.000e+00,0.000e+00,0.000e+00,0.000e+00], [1.539e+03,1.000e+00,0.000e+00,0.000e+00,1.000e+00,0.000e+00,0.000e+00,  4.000e+00,0.000e+00,1.000e+00,0.000e+00,0.000e+00,0.000e+00,1.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,1.000e+00], [1.540e+03,5.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  1.000e+00,0.000e+00,3.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,1.000e+00,0.000e+00,0.000e+00,2.000e+00], [1.541e+03,5.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  1.000e+00,0.000e+00,5.000e+00,0.000e+00,0.000e+00,0.000e+00,3.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,2.000e+00]

Sending message
[[1.633e+03,0.000e+00,0.000e+00,0.000e+00,1.000e+00,0.000e+00,0.000e+00,  2.000e+00,0.000e+00,2.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,2.000e+00], [1.634e+03,5.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  3.000e+00,0.000e+00,2.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,2.000e+00,0.000e+00,0.000e+00,0.000e+00], [1.635e+03,3.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,1.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00], [1.636e+03,8.000e+00,0.000e+00,0.000e+00,1.000e+00,0.000e+00,0.000e+00,  1.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,4.000e+00], [1.637e+03,6.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  3.000e+00,0.000e+00,3.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,1.000e+00]

Sending message
[[1.729e+03,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00], [1.730e+03,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  1.000e+00,0.000e+00,1.000e+00,0.000e+00,0.000e+00,0.000e+00,1.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00], [1.731e+03,2.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,0.000e+00,1.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00], [1.732e+03,1.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,2.000e+00], [1.733e+03,1.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  1.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,  0.000e+00,0.000e+00,0.000e+00,0.000e+00,2.000e+00]

KeyboardInterrupt: 